In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
list=[['f1','b','d','m',1],
     ['f1','a','b','c',0],
     ['f2','b','d','m',0],
     ['f2','a','b','c',0],
     ['f3','b','d','m',1],
     ['f3','a','b','c',1],
     ['f4','b','d','m',0],
      ['f4','a','b','c',0]]
df= spark.createDataFrame(list,['function_name','param1'])
df.show()

+-------------+------+------+------+------+
function_name|param1|param2|param3|result|
+-------------+------+------+------+------+
 f1| b| d| m| 1|
 f1| a| b| c| 0|
 f2| b| d| m| 0|
 f2| a| b| c| 0|
 f3| b| d| m| 1|
 f3| a| b| c| 1|
 f4| b| d| m| 0|
 f4| a| b| c| 0|
+-------------+------+------+------+------+

In [3]:
list=[['f1','a','b','c',1],
     ['f1','b','d','m',0],
     ['f2','a','b','c',0],
     ['f2','b','d','m',0],
     ['f3','a','b','c',1],
     ['f3','b','d','m',1],
     ['f4','a','b','c',0],
      ['f4','b','d','m',0]]
df= spark.createDataFrame(list,['function_name','param1','param2','param3','result'])
df.show()

+-------------+------+------+------+------+
function_name|param1|param2|param3|result|
+-------------+------+------+------+------+
 f1| a| b| c| 1|
 f1| b| d| m| 0|
 f2| a| b| c| 0|
 f2| b| d| m| 0|
 f3| a| b| c| 1|
 f3| b| d| m| 1|
 f4| a| b| c| 0|
 f4| b| d| m| 0|
+-------------+------+------+------+------+

In [4]:
w=Window().partitionBy("function_name").orderBy(F.col("param1"),F.col("param2"),F.col("param3"))
w1=Window().partitionBy("function_name")
df1=df.withColumn("result_list", F.collect_list("result").over(w)).withColumn("result2",F.row_number().over(w))\
.withColumn("result3",F.max("result2").over(w1))\
.filter(F.col("result2")==F.col("result3")).drop("param1","param2","param3","result","result2","result3")


In [5]:
df1.groupBy("result_list")\
   .agg(F.collect_list("function_name").alias("function_name_list")).show()

+-----------+------------------+
result_list|function_name_list|
+-----------+------------------+
 [1, 0]| [f1]|
 [1, 1]| [f3]|
 [0, 0]| [f2, f4]|
+-----------+------------------+